# SFTTrainer로 지도 미세 조정

이 노트북은 `trl` 라이브러리의 `SFTTrainer`를 사용하여 `HuggingFaceTB/SmolLM2-135M` 모델을 미세 조정하는 방법을 보여줍니다. 노트북 셀이 실행되어 모델을 미세 조정합니다. 다양한 데이터 세트를 시도하여 난이도를 선택할 수 있습니다.

<div style='background-color: lightblue; padding: 10px; border-radius: 5px; margin-bottom: 20px; color:black'>
    <h2 style='margin: 0;color:blue'>연습: SFTTrainer로 SmolLM2 미세 조정</h2>
    <p>Hugging Face 허브에서 데이터 세트를 가져와 모델을 미세 조정합니다. </p> 
    <p><b>난이도</b></p>
    <p>🐢 `HuggingFaceTB/smoltalk` 데이터 세트를 사용합니다.</p>
    <p>🐕 `bigcode/the-stack-smol` 데이터 세트를 시도하고 특정 하위 집합 `data/python`에서 코드 생성 모델을 미세 조정합니다.</p>
    <p>🦁 관심 있는 실제 사용 사례와 관련된 데이터 세트를 선택합니다.</p>
</div>

In [ ]:
# Google Colab에서 요구 사항 설치
# !pip install transformers datasets trl huggingface_hub

# Hugging Face에 인증

from huggingface_hub import login
login()

# 편의를 위해 허브 토큰을 HF_TOKEN으로 환경 변수에 만들 수 있습니다.

In [ ]:
# 필요한 라이브러리 가져오기
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
from trl import SFTConfig, SFTTrainer, setup_chat_format
import torch

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available() else "cpu"
)

# 모델 및 토크나이저 로드
model_name = "HuggingFaceTB/SmolLM2-135M"
model = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path=model_name
).to(device)
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=model_name)

# 채팅 형식 설정
model, tokenizer = setup_chat_format(model=model, tokenizer=tokenizer)

# 미세 조정을 저장하거나 업로드할 이름 설정
finetune_name = "SmolLM2-FT-MyDataset"
finetune_tags = ["smol-course", "module_1"]

# 기본 모델로 생성

여기서는 채팅 템플릿이 없는 기본 모델을 사용해 보겠습니다. 

In [ ]:
# 훈련 전에 기본 모델을 테스트합니다.
prompt = "Write a haiku about programming"

# 템플릿으로 형식 지정
messages = [{"role": "user", "content": prompt}]
formatted_prompt = tokenizer.apply_chat_template(messages, tokenize=False)

# 응답 생성
inputs = tokenizer(formatted_prompt, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens=100)
print("Before training:")
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

## 데이터 세트 준비

샘플 데이터 세트를 로드하고 훈련용으로 형식을 지정합니다. 데이터 세트는 각 입력이 프롬프트이고 출력이 모델의 예상 응답인 입력-출력 쌍으로 구성되어야 합니다.

**TRL은 모델의 채팅 템플릿을 기반으로 입력 메시지 형식을 지정합니다.** `role` 및 `content` 키가 있는 사전 목록으로 표시되어야 합니다.

In [ ]:
# 샘플 데이터 세트 로드
from datasets import load_dataset

# TODO: path 및 name 매개변수를 사용하여 데이터 세트 및 구성을 정의합니다.
ds = load_dataset(path="HuggingFaceTB/smoltalk", name="everyday-conversations")

In [ ]:
# TODO: 🦁 데이터 세트가 TRL이 채팅 템플릿으로 변환할 수 있는 형식이 아닌 경우 처리해야 합니다. [모듈](../chat_templates.md)을 참조하십시오.

## SFTTrainer 구성

`SFTTrainer`는 훈련 프로세스를 제어하는 다양한 매개변수로 구성됩니다. 여기에는 훈련 단계 수, 배치 크기, 학습률 및 평가 전략이 포함됩니다. 특정 요구 사항 및 계산 리소스에 따라 이러한 매개변수를 조정합니다.

In [ ]:
# SFTTrainer 구성
sft_config = SFTConfig(
    output_dir="./sft_output",
    max_steps=1000,  # 데이터 세트 크기 및 원하는 훈련 기간에 따라 조정합니다.
    per_device_train_batch_size=4,  # GPU 메모리 용량에 따라 설정합니다.
    learning_rate=5e-5,  # 미세 조정을 위한 일반적인 시작점입니다.
    logging_steps=10,  # 훈련 메트릭 로깅 빈도입니다.
    save_steps=100,  # 모델 체크포인트 저장 빈도입니다.
    evaluation_strategy="steps",  # 정기적으로 모델을 평가합니다.
    eval_steps=50,  # 평가 빈도입니다.
    use_mps_device=(
        True if device == "mps" else False
    ),  # 혼합 정밀도 훈련에 MPS를 사용합니다.
    hub_model_id=finetune_name,  # 모델에 고유한 이름을 설정합니다.
)

# SFTTrainer 초기화
trainer = SFTTrainer(
    model=model,
    args=sft_config,
    train_dataset=ds["train"],
    tokenizer=tokenizer,
    eval_dataset=ds["test"],
)

# TODO: 🦁 🐕 선택한 데이터 세트에 SFTTrainer 매개변수를 맞춥니다. 예를 들어 `bigcode/the-stack-smol` 데이터 세트를 사용하는 경우 `content` 열을 선택해야 합니다.`

## 모델 훈련

트레이너가 구성되었으므로 이제 모델 훈련을 진행할 수 있습니다. 훈련 프로세스에는 데이터 세트를 반복하고, 손실을 계산하고, 이 손실을 최소화하도록 모델의 매개변수를 업데이트하는 과정이 포함됩니다.

In [ ]:
# 모델 훈련
trainer.train()

# 모델 저장
trainer.save_model(f"./{finetune_name}")

In [ ]:
trainer.push_to_hub(tags=finetune_tags)

<div style='background-color: lightblue; padding: 10px; border-radius: 5px; margin-bottom: 20px; color:black'>
    <h2 style='margin: 0;color:blue'>보너스 연습: 미세 조정된 모델로 생성</h2>
    <p>🐕 기본 예제와 마찬가지로 미세 조정된 모델을 사용하여 응답을 생성합니다.</p>
</div>

In [ ]:
# 동일한 프롬프트에서 미세 조정된 모델 테스트

# 훈련 전에 기본 모델을 테스트합니다.
prompt = "Write a haiku about programming"

# 템플릿으로 형식 지정
messages = [{"role": "user", "content": prompt}]
formatted_prompt = tokenizer.apply_chat_template(messages, tokenize=False)

# 응답 생성
inputs = tokenizer(formatted_prompt, return_tensors="pt").to(device)

# TODO: 기본 예제와 마찬가지로 미세 조정된 모델을 사용하여 응답을 생성합니다.

## 💐 완료했습니다!

이 노트북은 `SFTTrainer`를 사용하여 `HuggingFaceTB/SmolLM2-135M` 모델을 미세 조정하는 단계별 가이드를 제공했습니다. 이러한 단계를 따르면 특정 작업을 보다 효과적으로 수행하도록 모델을 조정할 수 있습니다. 이 과정을 계속 진행하려면 다음 단계를 시도해 볼 수 있습니다.

- 더 어려운 난이도로 이 노트북을 시도해 보십시오.
- 동료의 PR을 검토하십시오.
- 문제 또는 PR을 통해 과정 자료를 개선하십시오.